# 03 Ray Tracing Basics

In order to better understand how to improve our sample applications performance, you should have a clear understanding of how ray tracing works, it's benefits, and caveats to watch out for. If you are already familiar with Ray Tracing, you can optionally skip this section.

## 03a - What is Ray Tracing
Ray Tracing is simply another type of rendering algorithm that like rasterization, takes geometry created by artists and displays them on the screen. While rasterization works by projecting geometry onto the screen using a virtual camera, Ray Tracing more closely simulates how the real world works by tracing rays that are similar to light photons. Ray tracing can be very simple (such as with ray casting, where only a single primary ray is used) or very complex (such as with path tracing where those rays bounce around the scene, interactin with various media).  

Lots of algorithms and effects can be created using Ray Tracing:
- Global Illumination (e.g. Path Tracing)
- Reflections
- Refraction
- Subsurface Scattering
- Ambient Occlusion
- Depth-of-Field (i.e. Camera Focus)

For a more thorough explanation, be sure to check out this blog post: https://blogs.nvidia.com/blog/2018/03/19/whats-difference-between-ray-tracing-rasterization/

The Ray Tracing Essentials video series by Eric Haines is also a great resource as Eric explains the concepts behind ray tracing in a very entertaining and easy to follow way:

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/gBPNO6ruevk" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## 03b - Concepts

Ray Tracing relies on a few key concepts. Imagine if you had to test a ray against every triangle in a world with millions of triangles. Not so smart right? This is where Acceleration Structures help; they ensure that triangle intersection operations are as efficient as possible by minimizing unnecessary tests. They can be static or dynamic (in the case of animating objects, such as characters).

<img src="images/image063.png" width="800"> 

Ray tracing involves shooting rays from every pixel on your screen and determining what they intersect with in your virtual world. In order to do this quickly, the world is subdivided into a top level and bottom level acceleration structure (also called TLAS and BLAS). The TLAS contains instances and transforms for the BLAS that it points to. The BLAS maintains an axis-aligned bounding box (or AABB) along with the geometry to render.

Ray Tracing in Vulkan requires a few new shader types:
<img src="images/image066.png" width="800"> 

- Ray Generation Shaders are where your rays are spawned. This is effectively the entry point for the TraceRays API call.
- Closest Hit Shaders are run when a ray intersects with scene geometry.
- Miss Shaders execute when a ray does not intersect with anything in your scene. You might put your skybox code here as an example.
- Intersection Shaders allow you to provide your own primitive intersection, for instance, spheres instead of AABBs.
- Any Hit Shaders run when there is a non-opaque intersection and you would like to modify the intersection result. This is useful to implement alpha-testing since you can permit rays to keep going.
- There's also a new shader type known as callable shaders that act like a subroutine, allowing you to dynamically execute a shader from within another shader (thus cutting down on the number of shader permutations needed and reducing the need for uber-shaders and excessive conditional branching).

Let's have a quick look in Nsight Graphics again. Open up the events list and look for vkCmdTraceRaysKHR. Using this inspector, we can verify that all of our shaders are properly configured.

<img src="images/ray-shaders-inspector.png" width="600"> 

Acceleration structures help to speed up ray traversal across the world since doing a coarse test on a simpler shape, in this case, the AABB, is much faster than testing for a triangle intersection directly. NVIDIA Turing and Ampere Architecture GPUs have specialized hardware to do this efficiently, but even so, it's very important that these structures are properly fitted to geometry in order to ensure as few AABBs are tested as possible.

<img src="images/image070.png" width="800">

It's your responsibility to supply Top and Bottom Level Acceleration Structures and to efficiently update them for dynamic objects.

One thing to keep in mind is that while rasterization has traditionally been very fast, ray tracing performance is comparible if proper tradeoffs are made. For instance, ray length is important as the longer a ray traverses the scene, the more objects must be considered. Reducing this may reduce visual fidelity, but likely at an imperceptable level while improving performance dramatically. It's up to you to experiment to find the best balance.


## 03c - EndeavorRTX

In a real world application, you're likely doing some combination of Ray Tracing and Rasterization. For instance, you may render all the characters in the world using Rasterization, then apply an Ambient Occlusion effect as a post-process that is then composited into the original rasterized image. In the future, we see all rendering being done using Ray Tracing, but there are certainly many challengers to overcome before we get there.

In our example application, we are doing pure Ray Tracing. Unlike Quake2 RTX (which Uri will show you in a bit), we aren't using Path Tracing to simulate real world lighting. Rather, I've implemented a few special effects to create a demonstration of the kinds of issues you may encounter.

Specifically, this application implements:
- Direct lighting
- Soft Shadows
    - Shadow Samples
- Ambient Occlusion
    - Occlusion Samples
- Reflections
- Realistic Camera Model (non-pinhole)
    - Camera Samples
- Fog & Atmospheric Scattering (i.e. Aerial Perspective)
- Weather (Rain)
- Skybox (lat/long map sampling)

All of these effects are tweakable, allowing you to see the performance impact in real time. This will allow you to more easily iterate using our tools since you can change scene characteristics and quickly see the results.

<img src="images/endeavrtx-1.png" width="1000">

<font color="yellow">
Feel free to take a moment to play around with the sample and modify the values to see how your performance (and visual quality) is affected.
</font>

## 03d - Quiz

In [3]:
from mcq import create_multipleChoice_widget
Q1 = create_multipleChoice_widget('Ray Tracing can better represent real world scenes because...', ['More complex shaders can be used', 'Rays better represent how light works', 'More geometry can be used'], 'Rays better represent how light works', '')
Q1

In [6]:
from mcq import create_multipleChoice_widget
Q1 = create_multipleChoice_widget('Which Ray Tracing Shader Type does _not_ exist?', ['Intersection Shader', 'Hit Shader', 'Geometry Shader', 'Anyhit Shader'], 'Geometry Shader', '')
Q1

Let's look at some common problems you may encounter and how you can use Nsight Graphics to solve them.

[Continue to the **Common Problems** section](04_common_problems.ipynb)